In [409]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import scipy.stats as stats
import statsmodels.stats.api as sts
import statsmodels.api as sms
import statsmodels.formula.api as statsmodels
import pylab
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import shapiro
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
pd.set_option('display.max_columns', None)

In [296]:
Train=pd.read_excel('Data_Train.xlsx')

In [297]:
Test=pd.read_csv('Test_set.csv')

In [298]:
Train['Status']='Train'
Test['Status']='Test'

In [299]:
M=Train.append(Test)
M=M.reset_index(drop=True)

In [300]:
M.head()

,Additional_Info,Airline,Arrival_Time,Date_of_Journey,Dep_Time,Destination,Duration,Price,Route,Source,Status,Total_Stops
0,No info,IndiGo,01:10 22 Mar,24/03/2019,22:20,New Delhi,2h 50m,3897.0,BLR → DEL,Banglore,Train,non-stop
1,No info,Air India,13:15,1/05/2019,05:50,Banglore,7h 25m,7662.0,CCU → IXR → BBI → BLR,Kolkata,Train,2 stops
2,No info,Jet Airways,04:25 10 Jun,9/06/2019,09:25,Cochin,19h,13882.0,DEL → LKO → BOM → COK,Delhi,Train,2 stops
3,No info,IndiGo,23:30,12/05/2019,18:05,Banglore,5h 25m,6218.0,CCU → NAG → BLR,Kolkata,Train,1 stop
4,No info,IndiGo,21:35,01/03/2019,16:50,New Delhi,4h 45m,13302.0,BLR → NAG → DEL,Banglore,Train,1 stop


In [301]:
for i in range(0,len(M['Date_of_Journey'])):
    M.loc[i,'Date_of_Journey']=M.loc[i,'Date_of_Journey'][:-5]

In [302]:
for i in range(0,len(M['Date_of_Journey'])):
    M.loc[i,'Date_of_Journey(Month)']=M.loc[i,'Date_of_Journey'][-2:]

In [303]:
M['Date_of_Journey(Month)'].unique()

array(['03', '05', '06', '04'], dtype=object)

In [304]:
M['Date_of_Journey(Month)']=M['Date_of_Journey(Month)'].astype(int)

In [305]:
M.drop('Date_of_Journey', axis=1, inplace=True)

In [306]:
M['Additional_Info'].replace('No Info','No info', inplace=True)

In [307]:
M.head()

,Additional_Info,Airline,Arrival_Time,Dep_Time,Destination,Duration,Price,Route,Source,Status,Total_Stops,Date_of_Journey(Month)
0,No info,IndiGo,01:10 22 Mar,22:20,New Delhi,2h 50m,3897.0,BLR → DEL,Banglore,Train,non-stop,3
1,No info,Air India,13:15,05:50,Banglore,7h 25m,7662.0,CCU → IXR → BBI → BLR,Kolkata,Train,2 stops,5
2,No info,Jet Airways,04:25 10 Jun,09:25,Cochin,19h,13882.0,DEL → LKO → BOM → COK,Delhi,Train,2 stops,6
3,No info,IndiGo,23:30,18:05,Banglore,5h 25m,6218.0,CCU → NAG → BLR,Kolkata,Train,1 stop,5
4,No info,IndiGo,21:35,16:50,New Delhi,4h 45m,13302.0,BLR → NAG → DEL,Banglore,Train,1 stop,3


In [308]:
for i in range(0,len(M['Dep_Time'])):
    M.loc[i,'Dep_Time']=M.loc[i,'Dep_Time'][:2]

In [309]:
M['Dep_Time']=M['Dep_Time'].astype(int)

In [310]:
for i in range(0,len(M['Dep_Time'])):
    if M.loc[i,'Dep_Time']>=0 and M.loc[i,'Dep_Time']<12:
        M.loc[i,'Dep_Time']='Morning'
    elif M.loc[i,'Dep_Time']>=12 and M.loc[i,'Dep_Time']<17:
        M.loc[i,'Dep_Time']='Noon'
    elif M.loc[i,'Dep_Time']>=17 and M.loc[i,'Dep_Time']<20:
        M.loc[i,'Dep_Time']='Eveing'
    elif M.loc[i,'Dep_Time']>=20 and M.loc[i,'Dep_Time']<24:
        M.loc[i,'Dep_Time']='Night'

In [311]:
for i in range(0,len(M['Arrival_Time'])):
    M.loc[i,'Arrival_Time']=M.loc[i,'Arrival_Time'][:2]

In [312]:
M['Arrival_Time'].unique()

array(['01', '13', '04', '23', '21', '11', '10', '05', '19', '22', '09',
       '12', '06', '08', '03', '20', '16', '18', '15', '14', '00', '02',
       '07', '17'], dtype=object)

In [313]:
M['Arrival_Time']=M['Arrival_Time'].astype(int)

In [314]:
for i in range(0,len(M['Dep_Time'])):
    if M.loc[i,'Arrival_Time']>=0 and M.loc[i,'Arrival_Time']<12:
        M.loc[i,'Arrival_Time']='Morning'
    elif M.loc[i,'Arrival_Time']>=12 and M.loc[i,'Arrival_Time']<17:
        M.loc[i,'Arrival_Time']='Noon'
    elif M.loc[i,'Arrival_Time']>=17 and M.loc[i,'Arrival_Time']<20:
        M.loc[i,'Arrival_Time']='Eveing'
    elif M.loc[i,'Arrival_Time']>=20 and M.loc[i,'Arrival_Time']<24:
        M.loc[i,'Arrival_Time']='Night'

In [315]:
M['Arrival_Time'].unique()

array(['Morning', 'Noon', 'Night', 'Eveing'], dtype=object)

In [316]:
M.rename({'Arrival_Time':'Arrival'}, axis=1, inplace=True)
M.rename({'Dep_Time':'Dep'}, axis=1, inplace=True)

In [317]:
M['Destination'].replace('Delhi','New Delhi', inplace=True)

In [318]:
M['Destination'].unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Hyderabad'],
      dtype=object)

In [319]:
M.head()

,Additional_Info,Airline,Arrival,Dep,Destination,Duration,Price,Route,Source,Status,Total_Stops,Date_of_Journey(Month)
0,No info,IndiGo,Morning,Night,New Delhi,2h 50m,3897.0,BLR → DEL,Banglore,Train,non-stop,3
1,No info,Air India,Noon,Morning,Banglore,7h 25m,7662.0,CCU → IXR → BBI → BLR,Kolkata,Train,2 stops,5
2,No info,Jet Airways,Morning,Morning,Cochin,19h,13882.0,DEL → LKO → BOM → COK,Delhi,Train,2 stops,6
3,No info,IndiGo,Night,Eveing,Banglore,5h 25m,6218.0,CCU → NAG → BLR,Kolkata,Train,1 stop,5
4,No info,IndiGo,Night,Noon,New Delhi,4h 45m,13302.0,BLR → NAG → DEL,Banglore,Train,1 stop,3


In [320]:
for i in range(0,len(M['Duration'])):
    if M.loc[i,'Duration'][-1]=='m':
        M.loc[i,'Duration_Minute']=M.loc[i,'Duration'][-3:-1]

In [321]:
M['Duration_Minute'].unique()

array(['50', '25', nan, '45', '30', ' 5', '15', '35', '10', '20', '55',
       '40', '5'], dtype=object)

In [322]:
M['Duration_Minute'].replace(np.nan,0,inplace=True)

In [323]:
for i in range(0,len(M['Duration'])):
    if M.loc[i,'Duration'][-1]=='m':
        M.loc[i,'Duration']=M.loc[i,'Duration'][:-3]

In [324]:
M['Duration'].replace('',' ',inplace=True)

In [325]:
M['Duration'].unique()

array(['2h ', '7h ', '19h', '5h ', '4h ', '15h ', '21h', '25h ', '13h ',
       '12h ', '26h ', '22h ', '23h', '20h ', '22h', '10h ', '6h ',
       '11h ', '11h', '8h ', '12h', '16h', '19h ', '3h ', '3h', '16h ',
       '15h', '25h', '27h ', '1h ', '5h', '14h ', '9h ', '10h', '21h ',
       '18h ', '18h', '17h ', '24h', '30h ', '14h', '23h ', '8h', '24h ',
       '28h ', '20h', '13h', '29h ', '9h', '37h ', '17h', '27h', '4h',
       '26h', '7h', '34h', '6h', '38h ', '30h', '28h', '35h ', '36h ',
       '35h', '47h', '33h ', '38h', '34h ', '29h', '32h ', '32h', '31h ',
       '42h', '39h', ' ', '41h ', '42h ', '40h ', '47h '], dtype=object)

In [326]:
for i in range(0,len(M['Duration'])):
    if M.loc[i,'Duration'][-1]==' ':
        M.loc[i,'Duration']=M.loc[i,'Duration'][:-1]

In [327]:
M['Duration'].unique()

array(['2h', '7h', '19h', '5h', '4h', '15h', '21h', '25h', '13h', '12h',
       '26h', '22h', '23h', '20h', '10h', '6h', '11h', '8h', '16h', '3h',
       '27h', '1h', '14h', '9h', '18h', '17h', '24h', '30h', '28h', '29h',
       '37h', '34h', '38h', '35h', '36h', '47h', '33h', '32h', '31h',
       '42h', '39h', '', '41h', '40h'], dtype=object)

In [328]:
for i in range(0,len(M['Duration'])):
    if M.loc[i,'Duration']=='':
        M.loc[i,'Duration']=''
    elif M.loc[i,'Duration']:
        M.loc[i,'Duration']=M.loc[i,'Duration'][:-1]

In [329]:
M['Duration'].unique()

array(['2', '7', '19', '5', '4', '15', '21', '25', '13', '12', '26', '22',
       '23', '20', '10', '6', '11', '8', '16', '3', '27', '1', '14', '9',
       '18', '17', '24', '30', '28', '29', '37', '34', '38', '35', '36',
       '47', '33', '32', '31', '42', '39', '', '41', '40'], dtype=object)

In [330]:
M['Duration'].replace('',0,inplace=True)
M['Duration_Minute'].replace('5',0,inplace=True)

In [331]:
M['Duration']=M['Duration'].astype(int)
M['Duration_Minute']=M['Duration_Minute'].astype(int)

In [332]:
for i in range(0,len(M['Duration'])):
    M.loc[i,'Total_Time']=M.loc[i,'Duration']*60+M.loc[i,'Duration_Minute']

In [333]:
M[(M['Airline']=='Air India') & (M['Destination']=='Hyderabad') & (M['Source']=='Mumbai') & (M['Total_Stops']=='2 stops') & (M['Route']=='BOM → GOI → PNQ → HYD')]

,Additional_Info,Airline,Arrival,Dep,Destination,Duration,Price,Route,Source,Status,Total_Stops,Date_of_Journey(Month),Duration_Minute,Total_Time
3821,No info,Air India,Noon,Night,Hyderabad,4,16172.0,BOM → GOI → PNQ → HYD,Mumbai,Train,2 stops,3,15,255.0
5309,No info,Air India,Noon,Night,Hyderabad,4,17327.0,BOM → GOI → PNQ → HYD,Mumbai,Train,2 stops,3,15,255.0
5942,No info,Air India,Noon,Morning,Hyderabad,11,16172.0,BOM → GOI → PNQ → HYD,Mumbai,Train,2 stops,3,50,710.0
6474,No info,Air India,Noon,Noon,Hyderabad,0,17327.0,BOM → GOI → PNQ → HYD,Mumbai,Train,2 stops,3,0,0.0
7456,No info,Air India,Noon,Morning,Hyderabad,11,16697.0,BOM → GOI → PNQ → HYD,Mumbai,Train,2 stops,3,50,710.0
8478,No info,Air India,Noon,Noon,Hyderabad,3,18797.0,BOM → GOI → PNQ → HYD,Mumbai,Train,2 stops,3,25,205.0


In [334]:
M.loc[6474,'Total_Time']=205
M.loc[13343,'Total_Time']=205

In [335]:
M.drop('Duration', inplace=True, axis=1)
M.drop('Duration_Minute', inplace=True, axis=1)

In [336]:
M['Total_Time']=M['Total_Time'].astype(int)

In [337]:
M['Total_Stops'].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [338]:
M['Total_Stops']=M['Total_Stops'].astype(str)

In [339]:
for i in range(0,len(M['Total_Stops'])):
    if M.loc[i,'Total_Stops']=='non-stop':
        M.loc[i,'Total_Stops']=0
    else:
        M.loc[i,'Total_Stops']=M.loc[i,'Total_Stops'][0]

In [340]:
M['Total_Stops'].unique()

array([0, '2', '1', '3', 'n', '4'], dtype=object)

In [347]:
M[M['Total_Stops']=='n']

,Additional_Info,Airline,Arrival,Dep,Destination,Price,Route,Source,Status,Total_Stops,Date_of_Journey(Month),Total_Time


In [345]:
M[(M['Airline']=='Air India') & (M['Destination']=='Cochin') & (M['Source']=='Delhi') & (M['Arrival']=='Morning') & (M['Dep']=='Morning') &(M['Total_Time']==1420) & (M['Price']==7480)]

,Additional_Info,Airline,Arrival,Dep,Destination,Price,Route,Source,Status,Total_Stops,Date_of_Journey(Month),Total_Time
1616,No info,Air India,Morning,Morning,Cochin,7480.0,DEL → MAA → COK,Delhi,Train,1,5,1420
2554,No info,Air India,Morning,Morning,Cochin,7480.0,DEL → MAA → COK,Delhi,Train,1,5,1420
4351,No info,Air India,Morning,Morning,Cochin,7480.0,DEL → MAA → COK,Delhi,Train,1,6,1420
7896,No info,Air India,Morning,Morning,Cochin,7480.0,DEL → MAA → COK,Delhi,Train,1,6,1420
9039,No info,Air India,Morning,Morning,Cochin,7480.0,NaN,Delhi,Train,n,5,1420


In [346]:
M.loc[9039,'Total_Stops']=1

In [350]:
M['Destination'].replace('New Delhi','Delhi', inplace=True)

In [354]:
M.drop('Route',axis=1,inplace=True)

In [357]:
M.head()

,Additional_Info,Airline,Arrival,Dep,Destination,Price,Source,Status,Total_Stops,Date_of_Journey(Month),Total_Time
0,No info,IndiGo,Morning,Night,Delhi,3897.0,Banglore,Train,0,3,170
1,No info,Air India,Noon,Morning,Banglore,7662.0,Kolkata,Train,2,5,445
2,No info,Jet Airways,Morning,Morning,Cochin,13882.0,Delhi,Train,2,6,1140
3,No info,IndiGo,Night,Eveing,Banglore,6218.0,Kolkata,Train,1,5,325
4,No info,IndiGo,Night,Noon,Delhi,13302.0,Banglore,Train,1,3,285


In [ ]:
M['Total_Stops']=M['Total_Stops'].astype(int)

In [531]:
M.to_excel('cleaned_file.xlsx')

In [376]:
Train=M[M['Status']=='Train']

In [377]:
Train.shape

(10683, 11)

In [378]:
Test=M[M['Status']=='Test']

In [379]:
Test.shape

(2671, 11)

In [380]:
Train.drop('Status', axis=1, inplace=True)
Test.drop('Status', axis=1, inplace=True)

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [381]:
Train.head()

,Additional_Info,Airline,Arrival,Dep,Destination,Price,Source,Total_Stops,Date_of_Journey(Month),Total_Time
0,No info,IndiGo,Morning,Night,Delhi,3897.0,Banglore,0,3,170
1,No info,Air India,Noon,Morning,Banglore,7662.0,Kolkata,2,5,445
2,No info,Jet Airways,Morning,Morning,Cochin,13882.0,Delhi,2,6,1140
3,No info,IndiGo,Night,Eveing,Banglore,6218.0,Kolkata,1,5,325
4,No info,IndiGo,Night,Noon,Delhi,13302.0,Banglore,1,3,285


In [382]:
from sklearn.preprocessing import StandardScaler

In [383]:
SC=StandardScaler()

In [384]:
Train['Total_Time']=SC.fit_transform(pd.DataFrame(Train['Total_Time']))

C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int32 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int32 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [387]:
Test['Total_Time']=SC.transform(pd.DataFrame(Test['Total_Time']))

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int32 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [389]:
Test.drop('Price',axis=1,inplace=True)

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [390]:
Test.head()

,Additional_Info,Airline,Arrival,Dep,Destination,Source,Total_Stops,Date_of_Journey(Month),Total_Time
10683,No info,Jet Airways,Morning,Eveing,Cochin,Delhi,1,6,0.023410
10684,No info,IndiGo,Morning,Morning,Banglore,Kolkata,1,5,-0.793811
10685,In-flight meal not included,Jet Airways,Eveing,Eveing,Cochin,Delhi,1,5,1.539700
10686,No info,Multiple carriers,Night,Morning,Cochin,Delhi,1,5,0.269561
10687,No info,Air Asia,Morning,Night,Delhi,Banglore,0,6,-0.931656


In [400]:
Train.drop('Additional_Info', axis=1, inplace=True)
Test.drop('Additional_Info', axis=1, inplace=True)

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [402]:
Train_dummy=pd.get_dummies(Train, drop_first=True)

In [405]:
Test_dummy=pd.get_dummies(Test,drop_first=True)

In [412]:
Train_dummy.drop('Airline_Trujet',axis=1,inplace=True)

In [415]:
Train_dummy.shape

(10683, 28)

In [416]:
Test_dummy.shape

(2671, 27)

In [418]:
x=Train_dummy.drop('Price',axis=1)
y=Train_dummy['Price']

### LR

In [556]:
lr=LinearRegression()

In [557]:
ypre=lr.fit(x,y).predict(Test_dummy)

In [558]:
pre=pd.DataFrame(ypre, columns={'Price'})

In [560]:
pre.to_excel('Prediction(LR).xlsx',index=False)

88.9

### RFR

In [571]:
from sklearn.ensemble import RandomForestRegressor

In [572]:
rfr=RandomForestRegressor()

In [573]:
ypre=rfr.fit(x,y).predict(Test_dummy)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [574]:
pre=pd.DataFrame(ypre, columns={'Price'})

In [575]:
pre.to_excel('Prediction(RF).xlsx',index=False)

88.94

### GBR

In [576]:
from sklearn.ensemble import GradientBoostingRegressor

In [577]:
gbr=GradientBoostingRegressor()

In [578]:
ypre=gbr.fit(x,y).predict(Test_dummy)

In [579]:
pre=pd.DataFrame(ypre, columns={'Price'})

In [580]:
pre.to_excel('Prediction(GB).xlsx',index=False)

88.907

### GSC(RFR)

In [581]:
from sklearn.model_selection import GridSearchCV

In [582]:
 param_grid = {"n_estimators":[10,20,30], "max_features":["auto", "sqrt", "log2"], "min_samples_split":[2,4,8], 
               "bootstrap":[True, False]}

In [583]:
grid = GridSearchCV(RandomForestRegressor(), param_grid, n_jobs=1, cv=5)

In [584]:
grid.fit(x,y)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=1,
       param_grid={'n_estimators': [10, 20, 30], 'max_features': ['auto', 'sqrt', 'log2'], 'min_samples_split': [2, 4, 8], 'bootstrap': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [585]:
grid.best_params_

{'bootstrap': True,
 'max_features': 'sqrt',
 'min_samples_split': 8,
 'n_estimators': 30}

In [586]:
rfr=RandomForestRegressor(bootstrap=True,max_features='sqrt',min_samples_split=8,n_estimators=30)

In [587]:
ypre=rfr.fit(x,y).predict(Test_dummy)

In [588]:
pre=pd.DataFrame(ypre, columns={'Price'})

In [589]:
pre.to_excel('Prediction(GSRF).xlsx',index=False)

89

### AdaBoost

In [590]:
from sklearn.ensemble import AdaBoostRegressor

In [591]:
abr=AdaBoostRegressor()

In [592]:
ypre=abr.fit(x,y).predict(Test_dummy)

In [593]:
pre=pd.DataFrame(ypre, columns={'Price'})

In [594]:
pre.to_excel('Prediction(Ada).xlsx',index=False)

89.4